
## Model Predictive Controllers
Model predictive controllers are a branch of controllers that minimizes the error between a desired trajectory and a predicted model response. In general, the optimization cost function is
\begin{equation}
\left(N_1, N_2, n_u\right) = E \left\lbrace    \sum_{j=N_1}^{N_2}\delta(j)\left[\hat y(t+j | t) - w(t+j)\right]^2+
\sum_{j=1}^{n_u} \lambda (j)\left[\Delta u(t+j-1)\right]^2 \right\rbrace
\end{equation}  
where $N_1$ and $N_2$ are the minimum and maximum cost horizons, $n_u$ is the control horizon, $\hat y$ is the future system
output on the considered horizon and $w$ is the determined reference signal.The control effort $\Delta u$ is penalized by
the parameter $\lambda$, and the predicted error is penalized by $\delta$.

## Dynamic Matrix Control

Dynamic Matrix Control (DMC) is a predictive controller that uses a step response to model the process. Since only the first $N$ terms are taken into account, it assumes that the process is stable and without integrators. The parameter $N$ is then the prediction horizon, and $n_u \le N$. Disturbances are taken to be the same as at instant $t$ along the entire predicted horizon and are defined as the measured output $y_m$ minus the output postulated by the model $\hat y$. The disturbance $\hat n(t)$ is then
\begin{equation}
\hat n(t+k | t) = \hat n(t | t) = y_m (t) - \hat y (t | t)
\end{equation}
where $k = 0,1\dots N-1$, and denotes a particular value along the prediction vector.
The prediction becomes
\begin{equation}
\hat y(t+k | t) = \sum_{i=1}^{k} a_i \Delta u(t+k-i) + \sum_{i=k+1}^{N} a_i \Delta u(t+k-i) + \hat n(t+k | t)
\end{equation}
where the first sum denotes the effect of past control moves and the second sum is due to future control actions. \ref{prediction} can be rewritten as
\begin{equation}
\hat y(t+k | t) = \hat y(t+k |t-1) + \mathbf{A} \Delta u
\end{equation}
where $\mathbf{A}$ is called a dynamic matrix.
In the single-input single-output (SISO) case, $\mathbf{A}$ contains the normalized step responses $a_i$ of the system model as
\begin{equation}
\mathbf{A} = \left[ \begin{matrix}
a_0 & 0 & \cdots & 0 \cr
a_1 & a_0 & \cdots & 0 \cr
\vdots & \vdots & \ddots & \vdots \cr
a_N & a_{N-1} & \cdots & a_{N-n_u+1}
\end{matrix} \right]_{\left(N\times n_u \right)}
\end{equation}
The optimal control move is found by introducing \ref{matrixADefinition} into \ref{costFunction}.
In the unconstrained predictive control algorithm, the resulting cost function is positive definite and therefore has only one
minimum, and can therefore be found by using a least square error approach. The resulting expression for the future control
moves $\Delta u$ is called the control law and is given by
\begin{equation}
\Delta u = \left(\mathbf{A}^T \mathbf{A} + \lambda I\right)^{-1}\mathbf{A}^T \left[w(t+k|t) - \hat{y}	(t+k|t)\right]
\end{equation}

In [ ]:
'''
SISO MPC Code
Meaghan Charest
Summer 2019
'''

# Import Libraries
import numpy as np
import matplotlib.pyplot as plt

#----------------------------
#### Important Variable ####
#----------------------------
#System Variables
Gain=14.2
Tau=0.72

#Control Variables
N=100
nu=3
dt = 0.05
Lambda = 1.01
alpha = 0.0
Max_ContAct=100.0
Min_ContAct=-100.0

#Simulation Variables
Step=np.zeros(N)
U=np.zeros(nu)
Uprev=np.zeros(nu)
Simulation_Length=60
Sim_time=np.zeros(Simulation_Length)
Control_Action = np.zeros(Simulation_Length)
Measurement=np.zeros(Simulation_Length)
Error=np.zeros(Simulation_Length)
SetVal=80.2
Setpoint =SetVal*np.ones(N)
Set=np.zeros(Simulation_Length)
yhat = np.zeros(N)

#Simulate a first Order step Response
for index in range (0,N-1):
    Step[index+1]=((Gain*1.0*dt)+(Tau*Step[index]))/(Tau+dt)

#Plots The Simulated First Order Plant
plt.plot(Step)
plt.title('First Order Response of the Plant to a Unit Step Input')
plt.ylabel('Response')
plt.xlabel('Sampling instant')
plt.show()
print(Step[0:10])


# Dynamic Matrix Construction
Dynamic_Matrix = np.zeros([N, nu])
for jj in range(0, nu):
    for ii in range(0, N-jj):
        Dynamic_Matrix[ii + jj, jj] = Step[ii]
plt.plot(Dynamic_Matrix)
#print(Dynamic_Matrix)

In [ ]:
#Plots The Dynamic Matix, Uncomment for checking
plt.plot(Dynamic_Matrix)
plt.title('Dynamic Matrix Examples')
plt.ylabel('Response')
plt.xlabel('Sampling instant')
plt.show()

# Weight the diagonal of the Matrix to ensure good conditionalaty
# for inversion.
Dynamic_Matrix_Transpose = np.transpose(Dynamic_Matrix)
DM_Transpose_DM = np.dot(Dynamic_Matrix_Transpose, Dynamic_Matrix)
for diag in range(0, nu - 1):
    DM_Transpose_DM[diag, diag] = DM_Transpose_DM[diag, diag] * Lambda
DTDinv = np.linalg.inv(DM_Transpose_DM)
DM = np.dot(DTDinv, Dynamic_Matrix_Transpose)

# Control Loop Simulation
for index in range (1,Simulation_Length):

    #Simulate Plant actuation and response
    Measurement[index]=((Gain*0.9*Control_Action[index-1]*dt)+(Tau*Measurement[index-1]))/(Tau+dt)

    #Model Correction
    phi = Measurement[index]-yhat[0]
    yhat=yhat+phi

    # Evaluate Error
    error = Setpoint - yhat


    # Solve for optimal addition to previous control move
    deltaU = np.dot(DM, error)
    U = Uprev + deltaU
    Control_Action[index] = U[0]

    #Limit Control Move
    for index3 in range(0,nu):
        if U[index3] < Min_ContAct:
            U[index3] = Min_ContAct
        if U[index3] > Max_ContAct:
            U[index3] = Min_ContAct

    #Revaluate delta U
    deltaU=U-Uprev
    #print(deltaU)

    #Evaluate Prediciton
    yhat=yhat + Dynamic_Matrix[:, 0]*deltaU[0]

    #Update Values
    Uprev=U

    #Move prediction forward
    yhat[:-1] = yhat[1:]

    #build time array for plotting
    Sim_time[index]=index*dt
    Error[index]= error[1]
    Set[index]=SetVal

#Plots The Control Response, Error and Control Actions
f, axarr = plt.subplots(3, sharex=True)
axarr[0].plot(Sim_time,Measurement)
axarr[0].plot(Sim_time,Set)
axarr[0].set_title('Controled Plant Response')
axarr[1].plot(Sim_time,Control_Action, color='k')
axarr[1].set_title('Controled Plant Control Moves')
axarr[2].plot(Sim_time,Error, color='r')
axarr[2].set_title('Controled Plant Error')
plt.show()

